In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import make_scorer

In [2]:
df = pd.read_csv("train_modified.gz", compression="gzip")

In [3]:
# CHANGE RANDOM STATE
df_bootstrapped = df.sample(n=len(df), replace=True, random_state=101)

In [4]:
# dtype category speeds up catboost by 200x compared to dtyoe object
df_bootstrapped.iloc[:,:21] = df_bootstrapped.iloc[:,:21].astype("category")
df_bootstrapped.iloc[:,24] = df_bootstrapped.iloc[:,24].astype("category")
df_bootstrapped.iloc[:,26] = df_bootstrapped.iloc[:,26].astype("category")

In [5]:
X_train = df_bootstrapped.drop(["click"],axis=1)
y_train = df_bootstrapped["click"]

In [6]:
categorical_f = ['hour', 'C1', 'banner_pos', 'site_id', 'site_domain','site_category','device_id','device_ip','device_model',
                 'device_type','device_conn_type', 'C14','C15','C16','C17','C18', 'C19','C20','C21',
                 'day_of_week', 'user', 'click_history']

In [7]:
catboost = CatBoostClassifier(loss_function='Logloss', cat_features=categorical_f, data_partition='FeatureParallel', 
                              bootstrap_type='Bernoulli', verbose=10)

param = {
    'iterations':Integer(100, 250), # on the low side to speed up computation (learning rate will adjust accordingly)
    'depth':Integer(1, 10),
    'random_strength':Real(1e-9, 10), # amount of randomness to use for scoring splits (used to prevent overfitting)
    #'bagging_temperature':Real(0.0, 1.0),
    'l2_leaf_reg':Real(0.001, 10000), # coefficient at the L2 regularization term (lambda)
    'scale_pos_weight':Real(1, 50), # weight for class 1 in binary classification
    'subsample':Real(0.5, 1),
    'colsample_bylevel':Real(0.5,1),
    'model_size_reg':Real(0.01, 1000), # model size regularization coefficient
    'leaf_estimation_iterations':[1,5] # how many steps are done in every tree when calculating leaf values (values recommendated in documentation)
}

LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True) 
    
opt = BayesSearchCV(catboost, param, scoring = LogLoss, n_iter=20, cv=3, random_state=101, verbose=1)

# executes bayesian optimization
opt.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 0.6740053	total: 1.39s	remaining: 5m 17s
10:	learn: 0.5372718	total: 8.38s	remaining: 2m 46s
20:	learn: 0.4688510	total: 16.9s	remaining: 2m 48s
30:	learn: 0.4293319	total: 28.6s	remaining: 3m 3s
40:	learn: 0.4094504	total: 38.4s	remaining: 2m 57s
50:	learn: 0.3961264	total: 46.1s	remaining: 2m 41s
60:	learn: 0.3881612	total: 57.4s	remaining: 2m 39s
70:	learn: 0.3838108	total: 1m 7s	remaining: 2m 31s
80:	learn: 0.3800006	total: 1m 20s	remaining: 2m 27s
90:	learn: 0.3775015	total: 1m 32s	remaining: 2m 20s
100:	learn: 0.3759183	total: 1m 43s	remaining: 2m 11s
110:	learn: 0.3744892	total: 1m 54s	remaining: 2m 3s
120:	learn: 0.3733439	total: 2m 6s	remaining: 1m 54s
130:	learn: 0.3726665	total: 2m 18s	remaining: 1m 44s
140:	learn: 0.3719511	total: 2m 31s	remaining: 1m 35s
150:	learn: 0.3714627	total: 2m 43s	remaining: 1m 25s
160:	learn: 0.3709652	total: 2m 54s	remaining: 1m 14s
170:	learn: 0.3696869	total: 3m 3s	remaining: 1m 3s
180:	learn: 0.3667649	total: 3m 12s	remaining: 52.1s

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 12.6min finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 0.6686808	total: 1.2s	remaining: 1m 58s
10:	learn: 0.5077893	total: 9.99s	remaining: 1m 20s
20:	learn: 0.4380550	total: 19.2s	remaining: 1m 12s
30:	learn: 0.4049844	total: 28.3s	remaining: 1m 2s
40:	learn: 0.3882222	total: 37.3s	remaining: 53.6s
50:	learn: 0.3790622	total: 46.5s	remaining: 44.7s
60:	learn: 0.3736320	total: 55.4s	remaining: 35.4s
70:	learn: 0.3697955	total: 1m 4s	remaining: 26.3s
80:	learn: 0.3673313	total: 1m 13s	remaining: 17.1s
90:	learn: 0.3657712	total: 1m 21s	remaining: 8.1s
99:	learn: 0.3645285	total: 1m 30s	remaining: 0us
0:	learn: 0.6686814	total: 1.12s	remaining: 1m 50s
10:	learn: 0.5078768	total: 9.99s	remaining: 1m 20s
20:	learn: 0.4383659	total: 19.1s	remaining: 1m 11s
30:	learn: 0.4044612	total: 28.2s	remaining: 1m 2s
40:	learn: 0.3881870	total: 37s	remaining: 53.2s
50:	learn: 0.3792292	total: 45.7s	remaining: 43.9s
60:	learn: 0.3738899	total: 54.6s	remaining: 34.9s
70:	learn: 0.3700105	total: 1m 3s	remaining: 25.9s
80:	learn: 0.3674677	total: 1m

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  5.1min finished


0:	learn: 0.6745345	total: 6.31s	remaining: 26m 12s
10:	learn: 0.5341978	total: 1m 15s	remaining: 27m 28s
20:	learn: 0.4492792	total: 2m 32s	remaining: 27m 46s
30:	learn: 0.3942119	total: 3m 48s	remaining: 26m 51s
40:	learn: 0.3582109	total: 4m 59s	remaining: 25m 28s
50:	learn: 0.3331977	total: 6m 25s	remaining: 25m 3s
60:	learn: 0.3156431	total: 7m 37s	remaining: 23m 36s
70:	learn: 0.3032966	total: 8m 56s	remaining: 22m 31s
80:	learn: 0.2948258	total: 10m 6s	remaining: 21m 5s
90:	learn: 0.2889365	total: 11m 21s	remaining: 19m 51s
100:	learn: 0.2850023	total: 12m 15s	remaining: 18m 4s
110:	learn: 0.2813018	total: 13m 21s	remaining: 16m 43s
120:	learn: 0.2789647	total: 14m 21s	remaining: 15m 18s
130:	learn: 0.2773274	total: 15m 15s	remaining: 13m 51s
140:	learn: 0.2757404	total: 16m 20s	remaining: 12m 37s
150:	learn: 0.2747559	total: 17m 16s	remaining: 11m 19s
160:	learn: 0.2733356	total: 18m 30s	remaining: 10m 13s
170:	learn: 0.2725277	total: 19m 21s	remaining: 8m 56s
180:	learn: 0.271

BayesSearchCV(cv=3,
              estimator=<catboost.core.CatBoostClassifier object at 0x7f6f4841cb50>,
              n_iter=20, random_state=101,
              scoring=make_scorer(log_loss, greater_is_better=False, needs_proba=True),
              search_spaces={'colsample_bylevel': Real(low=0.5, high=1, prior='uniform', transform='identity'),
                             'depth': Integer(low=1, high=10, prior='uniform', transform='identity'),
                             'iter...
                             'leaf_estimation_iterations': [1, 5],
                             'model_size_reg': Real(low=0.01, high=1000, prior='uniform', transform='identity'),
                             'random_strength': Real(low=1e-09, high=10, prior='uniform', transform='identity'),
                             'scale_pos_weight': Real(low=1, high=50, prior='uniform', transform='identity'),
                             'subsample': Real(low=0.5, high=1, prior='uniform', transform='identity')},
    

In [8]:
opt.best_params_

OrderedDict([('colsample_bylevel', 0.5),
             ('depth', 10),
             ('iterations', 250),
             ('l2_leaf_reg', 0.001),
             ('leaf_estimation_iterations', 1),
             ('model_size_reg', 0.01),
             ('random_strength', 10.0),
             ('scale_pos_weight', 1.0),
             ('subsample', 1.0)])

In [9]:
opt.best_score_

-0.2208491018906314

In [ ]:
test = pd.read_csv("test_modified.gz", compression="gzip")
X_test = test.drop(["click"],axis=1)
y_test = test["click"]
y_pred = opt.predict_proba(X_test)

### get the predict_proba

In [10]:
best_p_l = {
    'boosting_type':'gbdt', 
    'objective': 'binary', 
    'metric':'binary_logloss', 
    'bagging_freq': 5, 
    'max_depth': 7,
    'learning_rate': 0.11304216699488043,
    'feature_fraction': 0.5066204305086464,
    'bagging_fraction': 0.6657456066570288,
    'max_bin': 188,
    'n_estimators': 482,
    'num_leaves': 60,
    'min_sum_hessian_in_leaf':72
}

In [11]:
def convert_obj_to_int(self):
    
    object_list_columns = self.columns
    object_list_dtypes = self.dtypes
    new_col_suffix = '_int'
    for index in range(0,len(object_list_columns)):
        if object_list_dtypes[index] == object :
            self[object_list_columns[index]+new_col_suffix] = self[object_list_columns[index]].map( lambda  x: hash(x))
            self.drop([object_list_columns[index]],inplace=True,axis=1)
    return self

X_train = df.drop(["click"],axis=1)
y_train = df["click"]
X_train_hash = X_train.copy()
column_list = ['C1', 'banner_pos', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 
               'C19', 'C20', 'C21']
X_train_hash[column_list] = X_train_hash[column_list].astype('object')
X_train_hash = convert_obj_to_int(X_train_hash)

In [13]:
from sklearn.model_selection import KFold

logloss_l=[]
proba_lgb_li=[]
y_li = []
n=0

kf = KFold(n_splits=5,random_state=90,shuffle=False)

for train_idx, test_idx in kf.split(X_train_hash,y_train):
    X_tr,X_val=X_train_hash.iloc[train_idx],X_train_hash.iloc[test_idx]
    y_tr,y_val=y_train.iloc[train_idx],y_train.iloc[test_idx]
    
    import lightgbm as lgb
    lgb = lgb.LGBMClassifier(**best_p_l, loss_function='Logloss')
    lgb.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100, verbose=False)
    proba_lgb = lgb.predict_proba(X_val.values)[:, 1]
    proba_lgb_li.append(proba_lgb)
    logloss_l.append(log_loss(y_val, proba_lgb))
    print(n+1,logloss_l[n])
    n+=1
    
    y_li.append(y_val)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


[LightGBM] [Warning] Unknown parameter: loss_function
[LightGBM] [Warning] bagging_fraction is set=0.6657456066570288, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6657456066570288
[LightGBM] [Warning] feature_fraction is set=0.5066204305086464, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5066204305086464
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=72, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=72
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
1 0.4149564156559571
[LightGBM] [Warning] Unknown parameter: loss_function
[LightGBM] [Warning] bagging_fraction is set=0.6657456066570288, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6657456066570288
[LightGBM] [Warning] feature_fraction is set=0.5066204305086464, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5066204305086464
[LightGBM] [Warning] mi

In [14]:
best_p_l_8 = {
    'boosting_type':'gbdt', 
    'objective': 'binary', 
    'metric':'binary_logloss', 
    'bagging_freq': 2, 
    'max_depth': 9,
    'learning_rate': 0.037681961372348104,
    'feature_fraction': 0.6186329542584896,
    'bagging_fraction': 0.7686771918501543,
    'max_bin': 198,
    'n_estimators': 854,
    'num_leaves': 58,
    'min_sum_hessian_in_leaf':34,
    'lambda_l1': 8.027647813535458,
    'lambda_l2': 5.230523285313312,
    'min_data_in_leaf': 93,
    'min_split_gain': 0.03929273115755069
}

In [15]:
from sklearn.model_selection import KFold

logloss_l_8=[]
proba_lgb_li_8=[]
n=0

kf = KFold(n_splits=5,random_state=90,shuffle=False)

for train_idx, test_idx in kf.split(X_train_hash,y_train):
    X_tr,X_val=X_train_hash.iloc[train_idx],X_train_hash.iloc[test_idx]
    y_tr,y_val=y_train.iloc[train_idx],y_train.iloc[test_idx]
    
    import lightgbm as lgb
    lgb = lgb.LGBMClassifier(**best_p_l_8, loss_function='Logloss')
    lgb.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100, verbose=False)
    proba_lgb = lgb.predict_proba(X_val.values)[:, 1]
    proba_lgb_li_8.append(proba_lgb)
    logloss_l_8.append(log_loss(y_val, proba_lgb))
    print(n+1,logloss_l_8[n])
    n+=1

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


[LightGBM] [Warning] Unknown parameter: loss_function
[LightGBM] [Warning] feature_fraction is set=0.6186329542584896, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6186329542584896
[LightGBM] [Warning] lambda_l1 is set=8.027647813535458, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.027647813535458
[LightGBM] [Warning] bagging_fraction is set=0.7686771918501543, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7686771918501543
[LightGBM] [Warning] lambda_l2 is set=5.230523285313312, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.230523285313312
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=34, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=34
[LightGBM] [Warning] min_data_in_leaf is set=93, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=93
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
1 0.4152410106

In [ ]:
best_p_l_28 = {
    'boosting_type':'gbdt', 
    'objective': 'binary', 
    'metric':'binary_logloss', 
    'bagging_freq': 7, 
    'max_depth': 10,
    'learning_rate': 0.3,
    'feature_fraction': 0.8999999999999999,
    'bagging_fraction': 0.8999999999999999,
    'max_bin': 59,
    'n_estimators': 218,
    'num_leaves': 80,
    'min_sum_hessian_in_leaf':0,
    'lambda_l1': 1e-08,
    'lambda_l2': 3.3855221440653636,
    'min_data_in_leaf':26,
    'min_split_gain': 0.03132393135883699
}

In [ ]:
from sklearn.model_selection import KFold

logloss_l_28=[]
proba_lgb_li_28=[]
n=0

kf = KFold(n_splits=5,random_state=90,shuffle=False)

for train_idx, test_idx in kf.split(X_train_hash,y_train):
    X_tr,X_val=X_train_hash.iloc[train_idx],X_train_hash.iloc[test_idx]
    y_tr,y_val=y_train.iloc[train_idx],y_train.iloc[test_idx]
    
    import lightgbm as lgb
    lgb = lgb.LGBMClassifier(**best_p_l_28, loss_function='Logloss')
    lgb.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100, verbose=False)
    proba_lgb = lgb.predict_proba(X_val.values)[:, 1]
    proba_lgb_li_28.append(proba_lgb)
    logloss_l_28.append(log_loss(y_val, proba_lgb))
    print(n+1,logloss_l_28[n])
    n+=1

In [ ]:
best_p_c = {
    'iterations': 300,
    'depth': 8,
    'l2_leaf_reg': 30,
    'random_strength': 9.725165337630147,
    'scale_pos_weight':1.0,
    'bagging_temperature':1.0,
    'border_count':1.0,
    'learning_rate': 0.23151807420324574
}

In [ ]:
best_p_c_2 = {
    'colsample_bylevel':1.0,
    'iterations': 250,
    'depth': 10,
    'l2_leaf_reg': 1000,
    'leaf_estimation_iterations': 5
    'model_size_reg': 0.001
    'random_strength': 10.0,
    'scale_pos_weight':1.0,
    'subsample':1.0
}

In [ ]:
best_p_c_240 = {
    'colsample_bylevel':0.6,
    'iterations': 250,
    'depth': 10,
    'l2_leaf_reg': 1000,
    'leaf_estimation_iterations': 1
    'model_size_reg': 0.001
    'random_strength': 1e-09,
    'scale_pos_weight':1.0,
    'subsample':0.6
}

In [ ]:
best_p_c_101 = {
    'colsample_bylevel':0.5,
    'iterations': 250,
    'depth': 10,
    'l2_leaf_reg': 0.001,
    'leaf_estimation_iterations': 1
    'model_size_reg': 0.01
    'random_strength': 10,
    'scale_pos_weight':1.0,
    'subsample':1.0
}